# Lab 1: Pansharpening
 **ISAT, EPITA**
 
 - **Write your name here**
 - **Write your name here**

## Information on the lab session

### Introduction
In this lab you will work directly on this notebook.
You will be required to:
- develop parts of code
- run some tests and analyse the results
- submit a report (the notebook with your answers filled in)

You can work in groups of two.

### Work during the session
- Rename the notebook adding your names
- Fill in your name/s in the first cell of the notebook
- Start reading the notebook, answer the questions, run the code, and write your own when needed
- When you are done, export the notebook in `pdf` (File -> Export Notebook As -> PDF)
- Create a compressed file including the notebook in `pdf` and in `ipynb` (this file), name it "**ISAT_labX-Name1_Name2**" (with X the n. of the lab)
- Submit your notebook in Chamilo (see below)

### Submission
The submission is done by sending the compressed folder to `mauro.dalla-mura@gipsa-lab.grenoble-inp.fr`. One submission per group.

The submission can take place in two steps.
1. **[Mandatory]** Submit your work **at the end of the session**.
2. **[Optional]** If you want to answer to additional questions (e.g., optional ones), you can submit a second version **within one week**. This second version will give up to 5 points wrt to the first submission.

# 1. Introduction

Passive (optical) very high resolution (VHR) satellites are equipped with two sensors acquiring two images of complementary characteristics.
- PAN: Panchromatic image (high spatial resolution, 1 band)
- MS: Multispectral image (low spatial resolution, ~3-16 bands)

This lab session is about **pansharpening**, i.e., the sharpening of a MS using the higher spatial resolution details in the PAN.
This can be seen as a particular problem of data fusion since one would aim at combining in a single product the spatial details resolved by the PAN (but not present in the MS) and the several spectral bands of the MS image (against the single band of the PAN). With respect to the general problem of multisensor fusion, pansharpening could not require the challenging phase of spatial coregistration since typically the images are acquired simultaneously being the PAN and MS sensors both mounted on the same platform.

In this lab session we will use *simulated data* obtained by degrading the resolution of real acquisitions.
This allows to have both a PAN and MS but also a reference MS (the MS at the original resolution).

## Notation

- $\mathbf{P}$: PAN image
- $\mathbf{M}$: MS image
- $\mathbf{M}^*$: High resolution MS image used as reference for evaluation (in real case scenarios **we don't have this image**)
- $\mathbf{S}$: Result of pansharpening. $\mathbf{S}$ should be as close as possible to $\mathbf{M}^*$

**Note** The original image $\mathbf{M}$ is 4 times smaller than the PAN. In this lab we will consider an image MS that has been already upsampled (e.g., with bicubic interpolation) in order to have the same size of the PAN image.

**Question** Write in few lines what is the physical limitation for having a single sensor of both high spectral and high spatial quality.

*Your answer here*

# 1.1 Needed packages

This cells contains all the packages and functions that are needed for executing all the cells of this notebook. Please feel free to add new `import` statements to customize the code and to make new additional experiments. 

In [ ]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error
from skimage import img_as_float


from skimage.color import rgb2hsv, hsv2rgb


In [ ]:
# Helper functions
def check_img_dimensions(x, y):
    """Check if two images have the same dimensions"""
    assert x.shape==y.shape, "Images have not the same dimension"

def norm_minmax(x, lower, upper):
    """Normalize an array to a given bound interval"""

    x_max = np.max(x)
    x_min = np.min(x)

    m = (upper - lower) / (x_max - x_min)
    x_norm = (m * (x - x_min)) + lower

    return x_norm

def norm_std(x, m, s):
    """Normalize an array in order to have a given mean and std"""
    
    x_norm = s * (x - x.mean(axis=(0,1))) / x.std(axis=(0,1)) + m

    return x_norm

def norm_minmax_img(x, y):
    """Normalize an array to a given bound interval given by image y"""
    check_img_dimensions(x, y)

    y_max = np.max(y)
    y_min = np.min(y)

    x_norm = norm_minmax(x, y_min, y_max)
    return x_norm

def norm_std_img(x, y):
    """Normalize an image x in order to have a given mean and std from image y"""
    check_img_dimensions(x, y)

    if x.ndim > 2:
        x_norm = x
        for i in range(x.shape[2]):
            x_norm[:,:,i] = norm_std_img(x[:,:,i], y[:,:,i])
        return x_norm
    else:
        y_mean = y.mean(axis=(0,1))
        y_std = y.std(axis=(0,1))
    
        # print(y_mean, y_std)

        x_norm = norm_std(x, y_mean, y_std)
        return x_norm
                 
    
def img_stats(I):
    """Print image statistics"""
    if I.ndim > 2:
        for i in range(I.shape[2]):
            print('Band {:d}'.format(i))
            img_stats(I[:,:,i])
    else:
        print('Mean: %.3f' %I.mean(axis=(0,1)))
        print('Std: %.3f' %I.std(axis=(0,1)))
        print('Min: {:.3f}, Max: {:.3f}'.format(I.min(axis=(0,1)), I.max(axis=(0,1))))

def imshow_bands(I):
    """Display the bands of an image"""
    
    nbands = I.shape[2]
    plt.figure(figsize=(20, 6))
    for i in range(nbands):
        plt.subplot(100 + nbands*10 + i+1)
        plt.imshow(I[:,:,i])
        plt.colorbar()
        plt.title("Band %d" %i)
    plt.show()

    
def imshow_stretch(I, stretchval=0.1):
    """Show a grayscale or RGB image after performing a stretching of its dynamic"""
    nbins = 1000
    tol_low = stretchval
    tol_high = 1-stretchval
    Iadj = np.zeros(I.shape)
    
    n_bands = 1 if len(I.shape) < 3 else I.shape[2]
    for i in range(n_bands):
        tmp = I if n_bands == 1 else I[:,:,i]
        tmp = tmp - np.min(tmp.ravel())
        tmp = tmp / np.max(tmp.ravel())
        N, _ = np.histogram(tmp.ravel(), nbins)
        cdf = np.cumsum(N)/np.sum(N)  # cumulative distribution function
        ilow = np.where(cdf > tol_low)[0][0]
        ihigh = np.where(cdf >= tol_high)[0][0]
        ilow = (ilow - 1)/(nbins-1)
        ihigh = (ihigh - 1)/(nbins-1)
        li = ilow
        lo = 0
        hi = ihigh
        ho = 255
        shape = tmp.shape
        out = (tmp < li) * lo
        out = out + np.logical_and(tmp >= li, tmp < hi) * (lo + (ho - lo) * ((tmp - li) / (hi - li)))
        out = out + (tmp >= hi) * ho
        if n_bands == 1:
            Iadj = out
        else:
            Iadj[:,:,i] = out
    plt.imshow(Iadj.astype('uint8'))


# 1.2 Data loading and preparation

Check the size of each image `X` (num rows, num cols, num bands) by typing `X.shape`.


In [ ]:
# Load MATLAB .mat file
from scipy.io import loadmat
x = loadmat('./data/Strsbrg1.mat')  # -> You can change dataset here, there are others in the folder

M = x['MS'] # Multispectral image 
HM = x['HRMS'] # Panchromatic image
P = x['PAN'] # High resolution Multispectral image (Reference)

del x

sizeP = P.shape
sizeHM = HM.shape
sizeM = M.shape


print('size M:  ', M.shape)
print('size HM: ', HM.shape)
print('size P:  ', P.shape)

**Question** By knowing that the Ground Sampling Distance (GSD) of the PAN sensor is 0.8 m. What is the GSD of the original MS image?

*Your answer here*

## 1.3 Data visualization

This part of the session is devoted to the visualization and inspection of the images we have.


* Visualize the image data
   * Look at one band of an image `X` with the function `imshow_stretch` (e.g., as in Python indexing starts from zero, if you want to see the 3rd band
you should type: `imshow_stretch(X[:,:,2])`)
  

**Note**: this instruction should be followed by `plt.show()`.

In [ ]:
# PAN
plt.figure
imshow_stretch(P) 
plt.set_cmap('gray')  # warning: permanent! 
plt.show()
# plt.set_cmap('viridis')  # restore the old colormap for future plots

**Question** The MS contains four spectral bands three in the visible domain (centered on the red, green and blue wavelenghts) and one in the near infrared. Show each band separately and try to identify them. Which is the right order of the spectral bands for showing a **true color** image (an image that have "real looking" colors)? Below we have used a sequence (1,2,3) just as an example.

*Your answer here*

In [ ]:
# example:
bands = (1,2,3)

# MS
plt.figure()
imshow_stretch(M[:,:,bands]) # -> this is clearly not a "true color" composition

# High res MS -> This is a reference image, usually we don't have it!
plt.figure()
imshow_stretch(HM[:,:,bands])


In [ ]:
# Define the band order here


## 1.4 Validation
As we have a reference image HM we can then compare the results obtained by pansharpening with respect to it.
The goal is thus to have an image resulting from pansharpening which is as close as possible to HM.

As quality indexes we will use:
- Minimum Square Error (MSE). Best value 0
- Structure Similarity Index (SSIM). Best value 1
These two indexes are contained in the function `evaluate` defined below.

Test the indexes by comparing the MS image (upsampled) **M** and the reference **HM**. This will be a baseline as there is no fusion with the PAN image in this case. Pansharpening should produce images of greater quality (lower MSE and higher SSIM) than this one. 

In [ ]:
def evaluate(S, HM):
    """Evaluate how close is the image S to the reference HM"""
    mse_img = mean_squared_error(S, HM)
    ssim_img = ssim(S, HM, data_range=HM.max() - HM.min(), multichannel=True)

    print('MSE: %.3f' %mse_img)
    print('SSIM: %.3f' %ssim_img)

# Evaluate how close is the original image M with the reference HM
evaluate(M, HM)


# 2. Component Substitution
This part of the lab will be devoted to the so-called Component Substitution family of pansharpening techniques.

This family is based on the projection of the MS image into another space, on the substitution of the PAN with one or more components of the transformed image and then on the inverse transform in the original space.

<img src="FlowchartCS.png" alt="CS" style="width: 500px;"/>

## 2.1 Introduction

In more details, considering the MS in matrix format ($\mathbf{M} = \left[\mathbf{M}_1, \dots, \mathbf{M}_N  \right]^T$, with $\mathbf{M}_i$ the i-th channel of the MS in a row vector format) and denoting by $\mathbf{X}$ the MS transformed by a generic linear transformation $A$, the forward transformation can be written as:
\begin{equation}\label{eq:CSforward}
 \mathbf{X} =A\mathbf{M},
\end{equation}

The assumption is that the transformation $A$ separates the spatial structure from the spectral information in different components that are decoupled.  
For example, we can consider the case in which the first component carries the spatial information and this can be considered as a low-pass version of the PAN image ($\mathbf{I}$): $\mathbf{X} = [\mathbf{X_1}, \mathbf{X}_2, \dots \mathbf{X}_{N}]^T$ with $\mathbf{X_1} \approx \mathbf{I}$.
According to this representation, the fusion of the MS with PAN takes place by substituting the components carrying spatial information (in this case, the first component) with the PAN: 
$$\mathbf{Y} = [\mathbf{P}, \mathbf{X}_2, \dots \mathbf{X}_{N}]^T.$$

Then, the inverse transformation is applied on $\mathbf{Y}$ in order to obtain the pansharpened result:
\begin{equation}\label{eq:CSinverse}
\mathbf{S} = A^{-1} \mathbf{Y}.
\end{equation}

## 2.2 An illustrative example on RGB images

One of the first approaches that appeared considered as transformation $A$ the mapping from the RGB to the HSV color space.
The HSV aims at decoupling the spatial information which is contained in the component Value (V) to the spectral (color) components Hue (H) and Saturation (S).

Implement in the following the Component Substitution scheme presented before and evaluate the results.

**Hint** You can use `rgb2hsv` and `hsv2rgb` for converting from a color space to the other.


**Question** Show that the spatial and spectral components in the HSV color space are "more decoupled" with respect to in the original RGB space.

In [ ]:
# -> Your code here


## 2.3 Component Substitution in a general formulation


### General formulation
A more general formulation of the CS fusion scheme is given by
\begin{equation}
\mathbf{S}_{k}=\mathbf{M}_{k}+g_{k}\left(
\mathbf{P}-\mathbf{I}\right)  , \label{eq:genaralInjectionModel}%
\end{equation}
in which the subscript $k$ indicates the $k$-th band, $\mathbf{g}=[g_{1},\dots,g_{k}%
,\dots,g_{N}]$ is the vector of the **injection gains**, while, $\mathbf{I}$ is defined as follows:
\begin{equation}
\mathbf{I}= \sum_{i=1}^N w_{i}\mathbf{M}_{i}, \label{eq:linearPLP}%
\end{equation}
where the **weights** $\mathbf{w}=[w_{1},\dots,w_{i},\dots,w_{N}]$ measure the spectral overlap among the spectral bands and the panchromatic
image $\mathbf{P}$.

**[Optional] Question:**

Prove that the process reported above can be equivalently obtained by the component substitution in Sec. 2.1.
Is there an advantage in using this generic formuation with respect the scheme in Sec. 2.1?

**Question** Give a brief comment on the role of the coefficients $\mathbf{g}$ and $\mathbf{w}$ in the fusion process.

*Your answer here*


In [ ]:
# -> Your code here

## 2.4 Generalized HSI scheme
You will implement a scheme that is similar to the previous HSV transformation done on the RGB image but with the generalized scheme presented in Sec. 2.3

In particular you will consider that $w_i = 1/N$ for all $i$ and $g_k = 1 $ for all $k$.

In [ ]:
# -> Your code here


CS techniques can be affected if the range of the PAN and the MS images differ. This happens quite often as they are acquired by different sensors.
Try to perform a normalization of the PAN image in order to force it to have the same dynamic of $\mathbf{I}$. 

**Hint** Use the `norm_std_image` defined at the beginning of the file.

**Question** Compare the case with and without normalization.

In [ ]:
# -> Your code here


## 2.5 Multiplicative injection scheme
An alternative way to sharpen the MS with the details fo the PAN is based on a multiplicative scheme for injecting the spatial details.
The fused image is defined, for $k=1,\dots,N$, as:
\begin{equation}
\mathbf{S}_{k}=\mathbf{M}_{k}\cdot\frac{\mathbf{P}%
}{\mathbf{I}}. \label{eq:HPM}%
\end{equation}
being the operators applied point-wise on the matrices.

**Question** Implement this scheme and compare it with the previous additive injection model.

**[Optional] Question** Prove that this can be equivalently expressed as 
  \begin{equation}
  \mathbf{S}_{k}=\mathbf{M}_{k}+\frac{\mathbf{M}_{k}}{\mathbf{I}}\cdot(\mathbf{P}-\mathbf{I}),
  \end{equation}
 
  by setting, for $k=1,\dots,N$,
  \begin{equation}
  g_{k}=\frac{\mathbf{M}_{k}}{\mathbf{I}}.
  \end{equation}
  
**Hint** Elementwise division between two arrays `A` and `B` can be performed by `np.divide(A, B)` (or `A/B`). Elementwise multiplication can be done by `np.multiply(A, B)` (or `A*B`).

In [ ]:
# -> Your code here


### MultiResolution Analysis (MRA)

In the second class of pansharpening methods the contribution
of the PAN image to the fused product is achieved by calculating the
difference between $\mathbf{P}$ and a low pass version $\mathbf{P}_{L}$.
Namely the HRMS image is defined as
\begin{equation}
\mathbf{S}_{k}=\mathbf{M}_{k}+g_{k}\left(
\mathbf{P}-\mathbf{P}_{L}\right),\ \forall k=1,\dots,N \label{eq:MRA}%
\end{equation}
This paradigm has been denoted as *Amélioration de
la Résolution Spatiale par Injection de Structures* (*ARSIS*)
underlining that the purposes of these methods are the preservation of the
whole content of the LRMS image and the addition of further information
obtained from the PAN image, through spatial filtering.

According to definition the different approaches belonging to
this class are uniquely characterized by the algorithm employed for obtaining
the image $\mathbf{P}_{L}$ and by the combination coefficients $\left\{
g_{k}\right\}_{k=1,\ldots,N}$.

In a very general setting, $\mathbf{P}_{L}$ is achieved through an iterative
decomposition scheme (called MRA) aiming at constructing a sequence of signals with successively
reduced information (or \textit{pyramid}), through the repeated application of
the analysis operators. The type of decomposition constitutes a first
distinguishing feature of the various approaches and can range from very simple
methods based on a single level decomposition to more complex techniques relying upon a multilevel analysis.

In addition, the mathematical expression of $\mathbf{g}$ assumes different
forms in the literature, among which two options are the most commonly reported.
By setting all the coefficients equal to $1$ the \textit{additive} injection scheme is achieved:%
\begin{equation}
\mathbf{S}_{k}=\mathbf{M}_{k}+\left(  \mathbf{P}%
-\mathbf{P}_{L}\right)  ,\ \forall k=1,\dots,N. \label{eq:HPFModel}%
\end{equation}
The \textit{High Pass Modulation} (\textit{HPM}) method (also called \textit{multiplicative} injection scheme) is the other
well-known paradigm, defined through the expression
\begin{equation}
\mathbf{S}_{k}=\mathbf{M}_{k}+\frac{\mathbf{M}_{k}}{\mathbf{P}_{L}}\left(  \mathbf{P}-\mathbf{P}_{L}\right)
,\ \forall k=1,\dots,N, \label{eq:HPMModel}%
\end{equation}
in which the details are weighted by the ratio of the MS and the
PAN low pass version $\mathbf{P}_{L}$, with the aim of reproducing, in the
fused image, the local intensity contrast of the PAN. 

<img src="FlowchartMRA.png" alt="MRA" style="width: 500px;"/>


The flowchart is slightly simpler than for CS. In fact, as one can notice, it lacks the block aimed at estimating the weight vector $\mathbf{w}$, needed for the construction of the equivalent panchromatic image.
Accordingly, the required blocks are devoted to: i) Upsample the MS image to reach the panchromatic resolution; ii) Perform the pyramidal decomposition yielding the low pass version $\mathbf{P}_{L}$ of the PAN image; iii)
Compute the band dependent injection gains $\left\{  g_{k}\right\}
_{k=1,\ldots,N}$; iv) Inject the extracted details.


### Low pass filtering

The most direct implementation of the ARSIS concept consists in applying a
single Linear Time-Invariant (LTI) Low Pass Filter (LPF) $h_{LP}$ to the PAN image $\mathbf{P}$ for obtaining $\mathbf{P}_{L}$, thus resulting in the following formula
\begin{equation}
\mathbf{S}_{k}=\mathbf{M}_{k}+g_k\left(  \mathbf{P}%
-\mathbf{P}\ast h_{LP}\right)  ,\ \forall k=1,\dots,N, \label{eq:HPF2}%
\end{equation}
in which $\ast$ denotes the convolution operator.

This approach can be performed by employing several different
low pass filters $h_{LP}$, among which the most diffused uses Box, Gaussian and Laplacian masks. Furthermore, the simple additive injection scheme  or more elaborated alternatives can be exploited for injecting the spatial details. 

Among the possible couples of filters and coefficients, we chose the simplest scheme achievable by using the Box mask (i.e., mask with uniform weights, implementing an average filter) and additive injection, which leads to the pansharpening algorithm known as *High Pass Filtering* (*HPF*) method. 

The corresponding methodology employing the HPM injection scheme has been also proposed.

**Question**
- Implement and test this technique using a conventional average filter.
- Test both the additive and multiplicative schemes.

**[Optional] Question**
- Test different filters (Gaussian, laplacian,...)

**Question** The choice of the filter size is critical. Make a comparison when changing the size of the filter. Perform an analysis of the filters in the frequency domain and discuss the observed spectra with respect to the corresponding obtained results.
**Hint** If a filter is isotropic, you may consider only a 1D profile.


In [ ]:
# -> Your code here
